In [1]:
import os
import pandas as pd
import numpy as np
import random
import lightgbm as lgb
import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [3]:
df_train = pd.read_csv(FOLDER + 'train.csv')
df_test = pd.read_csv(FOLDER + 'test.csv')

In [4]:
try_test = '20190611_test_dist_cos.csv'
trained_model = '20190612_champs_models_lgb.pickle'
submission_name = '20190612submission.csv'

In [5]:
bond_types = df_train['type'].unique()
del df_train

In [6]:
df_test_dist = pd.read_csv(OUTPUT + try_test)

In [7]:
with open(OUTPUT + trained_model, 'rb') as f:
    models = pickle.load(f)

In [8]:
ids_arr = np.zeros(0)
pred_arr = np.zeros(0)
for i, b in enumerate(bond_types):
    df = df_test_dist.query('type == "{}"'.format(b))
    ids = df['id'].values
    ids_arr = np.hstack([ids_arr, ids])
    X = df[df.columns[5:]].values
    y_pred = models[i].predict(X).reshape(-1)
    pred_arr = np.hstack([pred_arr, y_pred])

In [9]:
se_ids = pd.Series(ids_arr, name='id', dtype='int')
se_pred = pd.Series(pred_arr, name='scalar_coupling_constant')
df_submission = pd.concat([se_ids, se_pred], axis=1)
df_submission_s = df_submission.sort_values('id')
df_submission_s.to_csv(OUTPUT + submission_name, index=False)